# Redes Bayesianas

In [15]:
import pandas as pd
from pomegranate import Node, DiscreteDistribution, ConditionalProbabilityTable, BayesianNetwork

## 1. Preparando o DataFrame

> CARREGAMENTO E PRÉ-PROCESSAMENTO DE DADOS

In [16]:
# Carrega o dataset
titanic = pd.read_csv('../Datasets/Titanic-Dataset.csv')

# Preenche os valores ausentes na coluna 'Age' com a mediana
median_age = titanic['Age'].median()
titanic['Age']= titanic['Age'].fillna(median_age)

# Remove as colunas irrelevantes para o modelo
columns_to_drop = ['PassengerId', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Name']
titanic = titanic.drop(columns_to_drop, axis=1)

# Mapeia os rótulos numéricos e de string para nomes descritivos
titanic['Survived'] = titanic['Survived'].map({0: "não_sobreviveu", 1: "sobreviveu"})
titanic['Pclass'] = titanic['Pclass'].map({1: "primeira_classe", 2: "segunda_classe", 3: "terceira_classe"})
titanic['Sex'] = titanic['Sex'].map({'male': "masculino", 'female': "feminino"})

print("DataFrame pronto. Colunas restantes:", titanic.columns.tolist())

DataFrame pronto. Colunas restantes: ['Survived', 'Pclass', 'Sex', 'Age']


## 2. Construindo a Rede Bayesiana

> CÁLCULO DE PROBABILIDADES E CONSTRUÇÃO DOS NÓS

In [17]:
# Cria as distribuições para os nós pais
sex_dist = DiscreteDistribution(titanic['Sex'].value_counts(normalize=True).to_dict())
pclass_dist = DiscreteDistribution(titanic['Pclass'].value_counts(normalize=True).to_dict())

# Formata a tabela de probabilidade condicional para o nó 'Survived'
prob_table_series = titanic.groupby(['Sex', 'Pclass'])['Survived'].value_counts(normalize=True)
prob_table_pomegranate = prob_table_series.reset_index(name='probability').values.tolist()

In [18]:
# Cria os nós da rede
sex_node = Node(sex_dist, name="Sex")
pclass_node = Node(pclass_dist, name="Pclass")
survived_node = Node(
    ConditionalProbabilityTable(
        prob_table_pomegranate,
        [sex_node.distribution, pclass_node.distribution]
    ), name="Survived"
)

## 3. Montando e Treinando o Modelo

In [19]:
# Cria o modelo da Rede Bayesiana
model = BayesianNetwork()

# Adiciona os nós e as arestas
model.add_states(sex_node, pclass_node, survived_node)
model.add_edge(sex_node, survived_node)
model.add_edge(pclass_node, survived_node)

model.bake()

## 4. Realizando Previsões (Inferência)

> REALIZAR INFERÊNCIA E PREDIÇÃO

In [20]:
def print_predictions(predictions):
    for node, prediction in zip(model.states, predictions):
        if isinstance(prediction, str):
            print(f"{node.name}: {prediction}")
        else:
            print(f"Probabilidade do nó '{node.name}':")
            for value, probability in prediction.parameters[0].items():
                print(f"\t{value}: {probability:.4f}")

## 5. Exemplos

* Previsão 1: Qual a probabilidade de sobrevivência para um passageiro do sexo masculino?

In [21]:
print("Previsão 1: Sabendo que o passageiro é do sexo masculino...")
predictions_1 = model.predict_proba({"Sex": "masculino"})
print_predictions(predictions_1)

Previsão 1: Sabendo que o passageiro é do sexo masculino...
Sex: masculino
Probabilidade do nó 'Pclass':
	terceira_classe: 0.5511
	primeira_classe: 0.2424
	segunda_classe: 0.2065
Probabilidade do nó 'Survived':
	não_sobreviveu: 0.8034
	sobreviveu: 0.1966


* Previsão 2: Qual a probabilidade de sobrevivência sabendo que o passageiro é da primeira classe?

In [22]:
print("Previsão 2: Sabendo que o passageiro é da primeira classe...")
predictions_2 = model.predict_proba({"Pclass": "primeira_classe"})
print_predictions(predictions_2)

Previsão 2: Sabendo que o passageiro é da primeira classe...
Probabilidade do nó 'Sex':
	masculino: 0.6476
	feminino: 0.3524
Pclass: primeira_classe
Probabilidade do nó 'Survived':
	não_sobreviveu: 0.4200
	sobreviveu: 0.5800


* Previsão 3: Qual a probabilidade de sobrevivência para um passageiro do sexo feminino?

In [23]:
print("Previsão 3: Sabendo que o passageiro é do sexo feminino...")
predictions_1 = model.predict_proba({"Sex": "feminino"})
print_predictions(predictions_1)


Previsão 3: Sabendo que o passageiro é do sexo feminino...
Sex: feminino
Probabilidade do nó 'Pclass':
	terceira_classe: 0.5511
	primeira_classe: 0.2424
	segunda_classe: 0.2065
Probabilidade do nó 'Survived':
	não_sobreviveu: 0.2996
	sobreviveu: 0.7004


* Previsão 4: Qual a probabilidade de sobrevivência para um passageiro do sexo feminino e da primeira classe?


In [24]:
print("Previsão 4: Sabendo que o passageiro é do sexo feminino e da primeira classe...")
predictions_1 = model.predict_proba({"Sex": "feminino", "Pclass": "primeira_classe"})
print_predictions(predictions_1)

Previsão 4: Sabendo que o passageiro é do sexo feminino e da primeira classe...
Sex: feminino
Pclass: primeira_classe
Probabilidade do nó 'Survived':
	não_sobreviveu: 0.0319
	sobreviveu: 0.9681


* Previsão 5: Qual a probabilidade de sobrevivência para um passageiro do sexo feminino e da terceira classe?


In [25]:
print("Previsão 5: Sabendo que o passageiro é do sexo feminino e da terceira classe...")
predictions_1 = model.predict_proba({"Sex": "feminino", "Pclass": "terceira_classe"})
print_predictions(predictions_1)

Previsão 5: Sabendo que o passageiro é do sexo feminino e da terceira classe...
Sex: feminino
Pclass: terceira_classe
Probabilidade do nó 'Survived':
	não_sobreviveu: 0.5000
	sobreviveu: 0.5000
